# Textil Flow Shop Optimization

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import random as rd
import simpy as sp
from scipy.stats import weibull_min
import utils

# Random Seed
rd.seed(12397)

# Parameters

NUM_MACHINES_STAGE_1 = 60
NUM_MACHINES_STAGE_2 = 2
NUM_MACHINES_STAGE_3 = 3

# Função de Weibull
utils.weibull(10,30)


In [62]:
# Machines

class Machine:
    def __init__(self, env,
                 prev1_maintenance_time,
                 prev2_maintenance_time,
                 corr_maintenance_time,
                 life_time,
                 tipo_manutencao = 2,
                 operating_time = 0):
        
        self.env = env
        self.prev1_maintenance_time = prev1_maintenance_time
        self.prev2_maintenance_time = prev2_maintenance_time
        self.corr_maintenance_time = corr_maintenance_time
        self.life_time = life_time
        self.tipo_manutencao = tipo_manutencao
        self.operating_time = operating_time
        self.working = True
        self.produce = env.process(self.produce())

    def produce(self):
        while True:
            if self.working:
                # Tempo de funcionamento da máquina antes da próxima quebra
                tempo_funcionamento = rd.expovariate(1.0 / 180)
                yield self.env.timeout(tempo_funcionamento)

                # Máquina quebrada
                self.funcionando = False
                tempo_quebra = self.env.now
                print(f"Máquina quebrou em {tempo_quebra}")

                # Tempo para consertar a máquina
                tempo_conserto = rd.expovariate(1.0 / self.tempo_conserto)
                yield self.env.timeout(tempo_conserto)

                # Máquina consertada
                self.funcionando = True
                tempo_conserto = self.env.now - tempo_quebra
                print(f"Máquina consertada em {self.env.now}. Tempo de conserto: {tempo_conserto}")


In [ ]:
class SistemaProdutivo:
    def __init__(self, env, num_machines, maintenance_time):
        self.env = env
        self.machines = [Machine(env, maintenance_time) for i in range(num_machines)]
        self.env.process(self.monitor())

    def monitor(self):
        while True:
            # Tempo entre checagens do sistema
            yield self.env.timeout(10)

            # Contagem de máquinas quebradas
            broken_machines = sum([not machine.working for machine in self.machines])
            print(f"{broken_machines} broken machines in {self.env.now}")


In [ ]:
dict(type='int', num_values=5)

: 